## Predicting customer satisfaction at Olist - a Brazilian e-commerce platform
DATA PREPARATION

In [141]:
import pandas as pd
import numpy as np
import os


In [142]:

customers = pd.read_csv('./Olist_sales_dataset/olist_customers_dataset.csv')
geolocation =  pd.read_csv('./Olist_sales_dataset/olist_geolocation_dataset.csv')
order_items =  pd.read_csv('./Olist_sales_dataset/olist_order_items_dataset.csv')
order_payments =  pd.read_csv('./Olist_sales_dataset/olist_order_payments_dataset.csv')
order_reviews =  pd.read_csv('./Olist_sales_dataset/olist_order_reviews_dataset.csv')
order_info =  pd.read_csv('./Olist_sales_dataset/olist_orders_dataset.csv')
products_info = pd.read_csv('./Olist_sales_dataset/olist_products_dataset.csv')
sellers = pd.read_csv('./Olist_sales_dataset/olist_sellers_dataset.csv')
product_category = pd.read_csv('./Olist_sales_dataset/product_category_name_translation.csv')

FileNotFoundError: [Errno 2] No such file or directory: './Olist_sales_dataset/olist_geolocation_dataset.csv'

### Customers

This dataset contains information about the customer id and their location in Brazil, which can be used to track the order delivery location and especially analyze each customer's purchasing experience.

Each customer is assigned a customer_unique_id, while a new customer_id is assigned for every time they make an order. This is a way to track repeating customers, which is crucial for an e-commerce platform.

### Drop unwanted columns & rename the state column to join with social-economic data later.

In [138]:
customers.drop(['customer_city', 'customer_unique_id'], axis=1, inplace=True)
customers.rename(columns={"customer_state": "state_code"}, inplace=True)

KeyError: "['customer_city', 'customer_unique_id'] not found in axis"

In [130]:
customers

,customer_id,customer_zip_code_prefix,state_code
0,06b8999e2fba1a1fbc88172c00ba8bc7,14409,SP
1,18955e83d337fd6b2def6b18a428ac77,9790,SP
2,4e7b3e00288586ebd08712fdd0374a03,1151,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,8775,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,13056,SP
...,...,...,...
99436,17ddf5dd5d51696bb3d7c6291687be6f,3937,SP
99437,e7b71a9017aa05c9a7fd292d714858e8,6764,SP
99438,5e28dfe12db7fb50a4b2f691faecea5e,60115,CE
99439,56b18e2166679b8a959d72dd06da27f9,92120,RS


### Geolocation

This dataset contains Brazilian zip codes and its lat/lng coordinates. This can be useful to analyze the shipping distances and do geospartial visualization.

In [131]:
geolocation

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP
...,...,...,...,...,...
1000158,99950,-28.068639,-52.010705,tapejara,RS
1000159,99900,-27.877125,-52.224882,getulio vargas,RS
1000160,99950,-28.071855,-52.014716,tapejara,RS
1000161,99980,-28.388932,-51.846871,david canabarro,RS


In [132]:
# Drop unwanted columns & drop duplicates to keep only one lat/long pair per zip_code prefix

## geolocation = geolocation.drop(['geolocation_city', 'geolocation_state'], axis=1)
## geolocation = geolocation.drop_duplicates(subset = ['geolocation_zip_code_prefix'],ignore_index=True)

-Create 2 new data frames to join with customer data later.

In [135]:
## geo_customer = geolocation.rename(columns={"geolocation_zip_code_prefix":"customer_zip_code_prefix",
                                                  #"geolocation_lat":"customer_lat",
                                                  #"geolocation_lng":"customer_lng"})
## geo_seller = geolocation.rename(columns={"geolocation_zip_code_prefix":"seller_zip_code_prefix",
                                                  #"geolocation_lat":"seller_lat",
                                                  #"geolocation_lng":"seller_lng"})
                




In [136]:
geolocation.rename(columns={"geolocation_state":"state_code"})

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,state_code
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP
...,...,...,...,...,...
1000158,99950,-28.068639,-52.010705,tapejara,RS
1000159,99900,-27.877125,-52.224882,getulio vargas,RS
1000160,99950,-28.071855,-52.014716,tapejara,RS
1000161,99980,-28.388932,-51.846871,david canabarro,RS


In [137]:
geolocation.to_csv(r'./Olist_sales_dataset/olist_geolocation_dataset_clean.csv',index=False)

### Order items

This dataset has information about the products in each order. Prices are in Brazilian currency - reais. Freight value is calculated according to each item measures and weight.

In [7]:
print(order_items.shape)
order_items.head(3)

(112650, 7)


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.9,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.0,17.87


- Shipping_limit_date indicates limit date for seller to handle the order over to the logistic partner. It's a step in shipping process, but it's the actual delivery time that affect customer satisfaction, so we'll drop this attribute.

In [8]:
order_items.drop('shipping_limit_date', axis=1, inplace=True)

- The order_item_id represents the item quantity in each order. Due to the lack of connection between the product, order and review data sets, it's not possible to link each product in a multi-item order to its own review. Therefore, we'll only keep single-item orders.

In [9]:
rows_to_keep = len(order_items.loc[order_items['order_item_id'] == 1,])
print('Number of single-item orders to keep: ', rows_to_keep)
print('Percentage: ', rows_to_keep/len(order_items)*100)

Number of single-item orders to keep:  98666
Percentage:  87.58632933865957


In [10]:
order_items_restricted = order_items.loc[order_items['order_item_id'] == 1,]
order_items_restricted.drop('order_item_id', axis=1, inplace=True)

/var/folders/9w/mnvxlv2d3dz56vfrxd2g3gj40000gn/T/ipykernel_3779/39375319.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  order_items_restricted.drop('order_item_id', axis=1, inplace=True)


## Order reviews

In [11]:
print(order_reviews.shape)
order_reviews.head(3)

(99224, 7)


,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24


In [12]:
print('Duplicates in order_id: ', order_reviews['order_id'].duplicated().sum())
print('Duplicates in review_id: ', order_reviews['review_id'].duplicated().sum())

Duplicates in order_id:  551
Duplicates in review_id:  814


- Take out dupplicates for same order_id by selecting the latest review based on review_answer_timestamp, as the data is already sorted by time. Then take out dupplicated reviews due to the issue mentioned above with multi-item orders & multi reviews.

In [13]:
order_reviews.drop_duplicates(subset=['order_id'], keep='last', ignore_index=True, inplace=True)
order_reviews.drop_duplicates(subset=['review_id'], keep=False, ignore_index=True, inplace=True)

- Create a new attribute to capture the time each customer takes to respond to survey after purchasing. Then drop the timestamp columns.

In [15]:
order_reviews.isna().sum() / len(order_reviews)

review_id                  0.000000
order_id                   0.000000
review_score               0.000000
review_comment_title       0.882476
review_comment_message     0.586504
review_creation_date       0.000000
review_answer_timestamp    0.000000
dtype: float64

- The columns with text reivews are in Portugese and have more than 50% of values missing, so we'll drop them.

In [16]:
order_reviews = order_reviews.drop(['review_id', 'review_comment_title', 'review_comment_message', 
                                    'review_creation_date', 'review_answer_timestamp'], axis=1)
order_reviews.head(3)

,order_id,review_score
0,73fc7af87114b39712e6da79b0a377eb,4
1,a548910a1c6147796b98fdf73dbeba33,5
2,f9e4b658b201a9f2ecdecbb34bed034b,5


## Order info

This data set tracks the order status and the shipping process.

In [63]:
print(order_info.shape)
order_info.head(3)

(99441, 8)


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00


In [64]:

order_info.isna().sum() / len(order_reviews)

order_id                         0.000000
customer_id                      0.000000
order_status                     0.000000
order_purchase_timestamp         0.000000
order_approved_at                0.001613
order_delivered_carrier_date     0.017969
order_delivered_customer_date    0.029882
order_estimated_delivery_date    0.000000
dtype: float64

-- Create aggregated attributes to track the shipping process.

In [65]:
# changing dtype to date
order_info.loc[:,'order_purchase_timestamp'] = pd.to_datetime(order_info['order_purchase_timestamp'],
                                                              format="%Y-%m-%d %H:%M:%S").dt.date
order_info.loc[:,'order_delivered_customer_date'] = pd.to_datetime(order_info['order_delivered_customer_date'],
                                                              format="%Y-%m-%d %H:%M:%S").dt.date
order_info.loc[:,'order_estimated_delivery_date'] = pd.to_datetime(order_info['order_estimated_delivery_date'],
                                                              format="%Y-%m-%d %H:%M:%S").dt.date


In [66]:
import numpy as np

# Example timedelta in nanoseconds
timedelta_ns = np.timedelta64(86400000000000, 'ns')  # 1 day in nanoseconds

# Convert to days
timedelta_days = timedelta_ns / np.timedelta64(1, 'D')

print(timedelta_days)

1.0


In [67]:
# track the estimated & actual days of delivery
order_info['estimated_days_of_delivery'] = (order_info['order_estimated_delivery_date'] 
                                            - order_info['order_purchase_timestamp']).astype('timedelta64[ns]')
order_info['actual_days_of_delivery'] = (order_info['order_delivered_customer_date'] 
                                            - order_info['order_purchase_timestamp']).astype('timedelta64[ns]')

- Create a year column as a key join with the socio-economic data later.

In [68]:
order_info['year'] = pd.to_datetime(order_info['order_purchase_timestamp'], format='%Y/%m/%d').dt.year

- Drop rows with order_status as canceled or unavailable. Drop irrelevant columns.

In [69]:
order_info = order_info[(~order_info['order_status'].isin(['canceled', 'unavailable']))]

cols_to_drop = ['order_approved_at', 'order_delivered_carrier_date', 
               'order_delivered_customer_date', 'order_estimated_delivery_date'] 
order_info = order_info.drop(cols_to_drop, axis=1)
order_info.head(3)

,order_id,customer_id,order_status,order_purchase_timestamp,estimated_days_of_delivery,actual_days_of_delivery,year
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02,16 days,8 days,2017
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24,20 days,14 days,2018
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08,27 days,9 days,2018


## Sellers

This data set contains seller id and their location information.

In [70]:
print(sellers.shape)
sellers.head(3)

(3095, 4)


,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ


In [71]:
sellers.drop(['seller_city', 'seller_state'], axis=1, inplace=True)

In [72]:
sellers.head(3)

,seller_id,seller_zip_code_prefix
0,3442f8959a84dea7ee197c632cb2df15,13023
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031


## Product info & categories

These data sets give details about Olist's products and their categories.

In [73]:
print(products_info.shape)
products_info.head(3)

(32951, 9)


,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0


- Join product info & category to get the category names in English, then drop the Portugese category names.

In [74]:
products = pd.merge(products_info, product_category, on='product_category_name', how='left')

products.drop('product_category_name', axis=1, inplace=True)

products = products.rename(columns={"product_name_lenght": "product_name_length", 
                                              "product_description_lenght": "product_description_length",
                                             "product_category_name_english": "product_category_name"})

In [75]:
products.isna().sum()

product_id                      0
product_name_length           610
product_description_length    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
product_category_name         623
dtype: int64

- Assign the missing values in product_category_name as 'not_reported'.

In [76]:
products['product_category_name'] = products['product_category_name'].fillna('not_reported')
products.head(3)

,product_id,product_name_length,product_description_length,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name
0,1e9e8ef04dbcff4541ed26657ea517e5,40.0,287.0,1.0,225.0,16.0,10.0,14.0,perfumery
1,3aa071139cb16b67ca9e5dea641aaa2f,44.0,276.0,1.0,1000.0,30.0,18.0,20.0,art
2,96bd76ec8810374ed1b65e291975717f,46.0,250.0,1.0,154.0,18.0,9.0,15.0,sports_leisure


## FINAL TABLE
Join data

In [112]:
final_df = order_items_restricted.merge(order_info, on = 'order_id', how = 'inner')
final_df = final_df.merge(products, on = 'product_id', how = 'left')
final_df = final_df.merge(order_reviews, on = 'order_id', how = 'inner')
final_df = final_df.merge(customers, on = 'customer_id', how = 'left')
final_df = final_df.merge(sellers, on = 'seller_id', how='left')
final_df = final_df.merge(geo_customer, on = 'customer_zip_code_prefix', how = 'left')
final_df = final_df.merge(geo_seller, on = 'seller_zip_code_prefix', how = 'left')


In [120]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98012 entries, 0 to 98011
Data columns (total 35 columns):
 #   Column                      Non-Null Count  Dtype          
---  ------                      --------------  -----          
 0   order_id                    98012 non-null  object         
 1   product_id                  98012 non-null  object         
 2   seller_id                   98012 non-null  object         
 3   price                       98012 non-null  float64        
 4   freight_value               98012 non-null  float64        
 5   customer_id                 98012 non-null  object         
 6   order_status                98012 non-null  object         
 7   order_purchase_timestamp    98012 non-null  object         
 8   estimated_days_of_delivery  98012 non-null  timedelta64[ns]
 9   actual_days_of_delivery     96353 non-null  timedelta64[ns]
 10  year                        98012 non-null  int32          
 11  product_name_length         96612 non-nul

## Treat missing values

In [113]:
final_df.isna().sum()

order_id                          0
product_id                        0
seller_id                         0
price                             0
freight_value                     0
customer_id                       0
order_status                      0
order_purchase_timestamp          0
estimated_days_of_delivery        0
actual_days_of_delivery        1659
year                              0
product_name_length            1400
product_description_length     1400
product_photos_qty             1400
product_weight_g                 16
product_length_cm                16
product_height_cm                16
product_width_cm                 16
product_category_name             0
review_id                         0
review_score                      0
review_comment_title          86595
review_comment_message        57861
review_creation_date              0
review_answer_timestamp           0
customer_unique_id                0
customer_zip_code_prefix          0
customer_city               

In [114]:
final_df[final_df.actual_days_of_delivery.isna()]['order_status'].value_counts()

order_status
shipped       1042
invoiced       311
processing     296
delivered        8
approved         2
Name: count, dtype: int64

For customer review, Olist sends survey to customers when the order is delivered or the estimated time of delivery is passed. So for those orders that don't have the delivery date yet but still have review scores, it means they haven't been delivered at the time of review.

Based on the average actual delivery days and each order's status, we estimate the extra time (on top of the estimated delivery time for each order) as follows:

- shipped: +5 days
- invoiced/processing/approved: + 10 days
- delivered: +1 day

In [115]:
import numpy as np

In [116]:
# track whether the delivery is late
final_df['late_delivery'] = (final_df['actual_days_of_delivery'] > final_df['estimated_days_of_delivery']).astype(int)
final_df['late_delivery'].value_counts()

late_delivery
0    91603
1     6409
Name: count, dtype: int64

## Latitude/longtitude coordin
Fill with the lat/lng pair of the closest zip_code_prefix.

In [117]:
null_cus_lat_index = final_df[final_df['customer_lat'].isnull()].index

for i in null_cus_lat_index:
    closest_zc = min(geo_customer['customer_zip_code_prefix'],
                     key=lambda x: abs(x - final_df.loc[i, 'customer_zip_code_prefix']))
    final_df.loc[i, ['customer_lat', 'customer_lng']] = geo_customer.loc[
        geo_customer['customer_zip_code_prefix'] == closest_zc, ['customer_lat', 'customer_lng']
    ].squeeze()  


In [118]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98012 entries, 0 to 98011
Data columns (total 35 columns):
 #   Column                      Non-Null Count  Dtype          
---  ------                      --------------  -----          
 0   order_id                    98012 non-null  object         
 1   product_id                  98012 non-null  object         
 2   seller_id                   98012 non-null  object         
 3   price                       98012 non-null  float64        
 4   freight_value               98012 non-null  float64        
 5   customer_id                 98012 non-null  object         
 6   order_status                98012 non-null  object         
 7   order_purchase_timestamp    98012 non-null  object         
 8   estimated_days_of_delivery  98012 non-null  timedelta64[ns]
 9   actual_days_of_delivery     96353 non-null  timedelta64[ns]
 10  year                        98012 non-null  int32          
 11  product_name_length         96612 non-nul

In [121]:
final_df.isna().sum()

order_id                          0
product_id                        0
seller_id                         0
price                             0
freight_value                     0
customer_id                       0
order_status                      0
order_purchase_timestamp          0
estimated_days_of_delivery        0
actual_days_of_delivery        1659
year                              0
product_name_length            1400
product_description_length     1400
product_photos_qty             1400
product_weight_g                 16
product_length_cm                16
product_height_cm                16
product_width_cm                 16
product_category_name             0
review_id                         0
review_score                      0
review_comment_title          86595
review_comment_message        57861
review_creation_date              0
review_answer_timestamp           0
customer_unique_id                0
customer_zip_code_prefix          0
customer_city               

- Export to a .csv file for the exploratory analysis.

In [119]:
final_df.to_csv(r'./Olist_sales_dataset/final_df.csv',index=False)

In [ ]:
import pandas as pd

# Example DataFrame
data = {
    'order_status': ['shipped', 'delivered', 'others'],
    'estimated_days_of_delivery': [5, 3, 7],
    'actual_days_of_delivery': [None, None, None]
}

final_df = pd.DataFrame(data)

# Define 'n' as the factor you want to use for adjustments
n = 1

# Define filters for different order statuses
shipped = ['shipped']
delivered = ['delivered']
others = ['others']

# Create boolean filters
filt_shipped = (final_df['actual_days_of_delivery'].isna() & final_df['order_status'].isin(shipped))
filt_delivered = (final_df['actual_days_of_delivery'].isna() & final_df['order_status'].isin(delivered))
filt_others = (final_df['actual_days_of_delivery'].isna() & final_df['order_status'].isin(others))

# Convert the columns to appropriate data types (Timedelta)
final_df['estimated_days_of_delivery'] = pd.to_timedelta(final_df['estimated_days_of_delivery'], unit='D')
final_df['actual_days_of_delivery'] = pd.to_timedelta(final_df['actual_days_of_delivery'], unit='D')

# Fill in values with extra time accordingly using the + operator
final_df.loc[filt_shipped, 'actual_days_of_delivery'] = final_df.loc[filt_shipped, 'estimated_days_of_delivery'] + pd.Timedelta(days=5)
final_df.loc[filt_delivered, 'actual_days_of_delivery'] = final_df.loc[filt_delivered, 'estimated_days_of_delivery'] + pd.Timedelta(days=1)
final_df.loc[filt_others, 'actual_days_of_delivery'] = final_df.loc[filt_others, 'estimated_days_of_delivery'] + pd.Timedelta(days=10)

# Convert the result back to numeric (days) if needed
final_df['estimated_days_of_delivery'] = final_df['estimated_days_of_delivery'].dt.days
final_df['actual_days_of_delivery'] = final_df['actual_days_of_delivery'].dt.days

print(final_df)
